In [9]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)

es = Elasticsearch(
    "http://192.168.1.230:30920",
    verify_certs=False,
    request_timeout=60,
    retry_on_timeout=True
)
index_name = "noms_prenoms"

df = pd.read_parquet("nom_prenoms_overlay_vf_clean.parquet")

def generate_docs(df):
    for i, row in df.iterrows():
        yield {
            "_index": index_name,
            "_id": i,
            "_source": row.to_dict()
        }
        
# Création de l'index avec 4 shards et 1 réplique
if not es.indices.exists(index=index_name):
    es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 4,
                "number_of_replicas": 1
            }
        }
    )


INFO:elastic_transport.transport:HEAD http://192.168.1.230:30920/noms_prenoms [status:200 duration:0.123s]


In [ ]:
try:
    helpers.bulk(es, generate_docs(df))
    print(f"{len(df)} documents insérés dans l'index '{index_name}'")
except Exception as e:
    print("Erreur lors de l'insertion bulk :", e)


INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.350s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.091s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.085s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.079s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.073s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.082s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.078s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.096s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.091s]
INFO:elastic_transport.transport:PUT http://192.168.1.230:30920/_bulk [status:200 duration:0.075s]
INFO:elast

1879306 documents insérés dans l'index 'noms_prenoms'
